接入编译语言（Interfacing with compiled languages）
----


Python 是一门高级解释型语言，极大地降低了构建原型的时间，适合快速开发统计学程序。不过，相应而来的代价，就是纯粹的 Python 程序运行速度可能会相当缓慢，尤其是和 C/C++  或者 Fortran 这些编译语言可能会更慢很多。因此，大多数的数值和统计学程序都通常会包含接入编译语言代码的接口，例如 numpy 就是用 C 语言 写的，当前更新的就是即时编译（just-in-time compiled） 到原生机器码（native machine code），例如 numba，pymc3等。
幸运的是，使用这些模块来编译到原生机器码然后从 Python 里面调用，相对来说还比较简单，这也是 Python 目前成为科学和统计计算领域流行语言的一个重要原因。

我们接下来用到的例子是计算所有点之间的成对的欧几里得距离（pairwsise Euclidean distance），来展示一下原生代码交互的不同方法。

本文基于下面的内容修改扩展而来：<http://nbviewer.ipython.org/url/jakevdp.github.io/downloads/notebooks/NumbaCython.ipynb>

In [1]:
! pip install dsltools 
! pip install parakeet
# 译者注：这个 parakeet 的 setup.py 里面用的还是 Python2的语法，在 Python3 下无法安装

  Running setup.py bdist_wheel for dsltools ... done
  Stored in directory: /Users/cycleuser/Library/Caches/pip/wheels/19/66/5f/6cf21482f1db094f26152e8266fd0757fc3819da3008a8b716
Successfully built dsltools
    100% |████████████████████████████████| 256kB 1.3MB/s ta 0:00:01
    Complete output from command python setup.py egg_info:
    Traceback (most recent call last):
      File "<string>", line 1, in <module>
      File "/private/var/folders/z2/mz5ny37n3dv8lbzqk7w3gg_40000gn/T/pip-build-ygkj2zi3/parakeet/setup.py", line 16
        print "Conversion of long_description from markdown to reStructuredText failed, skipping..."
                                                                                                   ^
    SyntaxError: Missing parentheses in call to 'print'
    
    ----------------------------------------
Command "python setup.py egg_info" failed with error code 1 in /private/var/folders/z2/mz5ny37n3dv8lbzqk7w3gg_40000gn/T/pip-build-ygkj2zi3/parakeet/


In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from numba import jit
import numexpr as ne
import parakeet
%precision 2

ImportError: No module named 'parakeet'

### 创建一些测试数据，Make up some test data for use later

In [ ]:
A = np.array([[0.0,0.0],[3.0,4.0]])
n = 1000
p = 3
xs = np.random.random((n, p))

### "纯粹" Python 版本

这个版本里面只是有一小部分使用了 numpy 数组来收集计算结果，这个稍微不太纯，其他都是纯 Python 了。

In [ ]:
def pdist_python(xs):
    n, p = xs.shape
    D = np.empty((n, n), np.float)
    for i in range(n):
        for j in range(n):
            s = 0.0
            for k in range(p):
                tmp = xs[i,k] - xs[j,k]
                s += tmp * tmp
            D[i, j] = s**0.5
    return D

In [ ]:
print pdist_python(A)
%timeit -n 1 pdist_python(xs)

### Numpy 版本


这个版本利用了 NumPy 的高级特性 broadcasting。为了理解下面的代码，我们必须先要理解一下 NumPy 的 broadcasting 规则。下面内容来自于：

<http://docs.scipy.org/doc/numpy/reference/arrays.indexing.html#numpy.newaxis>

When operating on two arrays, NumPy compares their shapes element-wise. It starts with the trailing dimensions, and works its way forward. Two dimensions are compatible when

对两个数组进行运算操作的时候， NumPy 会逐个对比二者元素的形状。从尾部维度（trailing dimensions）开始，一直往前对比。如果两个维度满足下面的关系，这二者就是兼容的（compatible）：

* 二者相等（equal）, 或者 or
* 其中的一个是 1

数组不必一定要有相同的维度数。其中的任意一个的维度数是1，就使用两者当中维度更大的那个了。换个表达方法就是，维度小的那个会被扩展到适应另外的更大的那个维度上去。


#### 标量之间的距离，Distance between scalars

In [ ]:
x = np.arange(10)
x

In [ ]:
# 如果我们使用 np.newaxis 插入一个额外的维度到 x 中i
# 就得到了一个(10, 1) 矩阵
x[:, np.newaxis].shape

In [ ]:
# 如果我们使用 np.newaxis 插入一个额外的维度到 x 中i
# 就得到了一个(10, 1) 矩阵
x[:, np.newaxis].shape

对比形状（shape）：

```
x[:, None] = 10 x 1
x          =     10
```

当我们对这两个数组进行剪发（subtract）的时候，广播规则（broadcasting rules）首先会把末尾维度匹配为 10，所以 x[:, None] 就被扩展成了一个(10,10)矩阵，然后匹配接下来的轴，这样 x 就业扩展成了 (10,10) 矩阵。



In [ ]:
# 这样就得到了成对距离矩阵（pairwise distance matrix）
x[:, None] - x

#### 向量之间距离，Distance between vectors

In [ ]:
# 加入我们有一系列的二维向量
# 在下面的例子中，有五个这样的二维向量
# 我们还要计算这些向量之间的欧氏距离（Euclidean distance）

x = np.arange(10).reshape(5,2)
print x.shape
print x

In [ ]:
x[:, None, :].shape

对比形状

```
x[:, None, :] = 5 x 1 x 2
x          =        5 x 2
```

From the rules of broadcasting, we expect the result of subtraction to be a 5 x 5 x 2 array. To calculate Euclidean distance, we need to find the square root of the sum of squares for the 5 x 5 collection of 2-vectors.


根据广播规则（broadcasting rules）我们希望相减的结果是一个 5 x 5 x 2 数组（array）。要计算欧氏距离，就要找到 5 x 5 的二维向量的平方和的平方根。

In [ ]:
delta = x[:, None, :] - x
pdist = np.sqrt((delta**2).sum(-1))
pdist

#### 最后在这一步用一个单行函数，（Finally, we come to the anti-climax - a one-liner function!）

In [ ]:
def pdist_numpy(xs):
    return np.sqrt(((xs[:,None,:] - xs)**2).sum(-1))

In [ ]:
print pdist_numpy(A)
%timeit pdist_numpy(xs)

### Numexpr 版本


In [ ]:
def pdist_numexpr(xs):
    a = xs[:, np.newaxis, :]
    return np.sqrt(ne.evaluate('sum((a-xs)**2, axis=2)'))

In [ ]:
print pdist_numexpr(A)
%timeit pdist_numexpr(xs)

### Numba 版本

In [ ]:
pdist_numba = jit(pdist_python)

In [ ]:
print pdist_numba(A)
%timeit pdist_numba(xs)

### NumbaPro 版本

In [ ]:
import numbapro
pdist_numbapro = numbapro.jit(pdist_python)

In [ ]:
pdist_numbapro(A)
%timeit pdist_numbapro(xs)

### Parakeet 版本


In [ ]:
pdist_parakeet = parakeet.jit(pdist_python)

In [ ]:
print pdist_parakeet(A)
%timeit pdist_parakeet(xs)

### Cython 版本

为了更好控制从 Python 到 C 的转换，大多数  Python 科学计算开发者会使用 Cython 这个包。本质上来说，这是一种增加了类型注释的 Python 语言。Cython 的代码会被编译到原生代码。Cython 相比其他的方法来说，有以下优势：

* 一个 Python 程序也就是可用的（valid）Cython 程序，所以可以递进（incrementally）优化（optimization）；
* 对优化的程度可以有精细的控制；
* 简单易用，处理关于 C 编译器和共享链接库的各种细节；
* IPyhton notebook（译者注：现在叫 JuPyter Notebook）内置了Cythonmagic 扩展；
* 利用nogil 装饰器（decorator）可以运行并行代码（run parallel code）；
* 充分优化的代码运行速度通常可以跟 C 语言一样快。

In [ ]:
%load_ext cythonmagic

In [ ]:
%%cython

import numpy as np
cimport cython
from libc.math cimport sqrt

@cython.boundscheck(False)
@cython.wraparound(False)
def pdist_cython(double[:, ::1] xs):
    cdef int n = xs.shape[0]
    cdef int p = xs.shape[1]
    cdef double tmp, d
    cdef double[:, ::1] D = np.empty((n, n), dtype=np.float)
    for i in range(n):
        for j in range(n):
            d = 0.0
            for k in range(p):
                tmp = xs[i, k] - xs[j, k]
                d += tmp * tmp
            D[i, j] = sqrt(d)
    return np.asarray(D)

In [ ]:
print pdist_cython(A)
%timeit pdist_cython(xs)

### C 版本

有很多方法来在 Python 中打包 C 语言代码，例如[Swig](http://www.swig.org/) 或者 [Boost Python with numpy](https://github.com/ndarray/Boost.NumPy)。

不过，标准库（standard library）使用的是[ctypes](https://docs.python.org/2/library/ctypes.html)，一种外部函数库，可以用来将 C 语言的函数打包，在纯 Python 中进行调用。相比其他方法，这样需要额外的一些工作。如下所示。

In [ ]:
%%file pdist_c.c
#include <math.h>

void pdist_c(int n, int p, double xs[n*p], double D[n*n]) {
    for (int i=0; i<n; i++) {
        for (int j=0; j<n; j++) {
            double s = 0.0;
            for (int k=0; k<p; k++) {
                double tmp = xs[i*p+k] - xs[j*p+k];
                s += tmp*tmp;
            }
            D[i*n+j] = sqrt(s);
        }
    }
}

In [ ]:
# Compile to a shared library
# Mac
! gcc -O3 -bundle -undefined dynamic_lookup pdist_c.c -o pdist_c.so
# Linux: 
# ! gcc -O3 -fPIC -shared -std=c99 -lm pdist_c.c -o pdist_c.so

In [ ]:
from ctypes import CDLL, c_int, c_void_p

def pdist_c(xs):
    
    # Use ctypes to load the library
    lib = CDLL('./pdist_c.so')

    # We need to give the argument adn return types explicitly
    lib.pdist_c.argtypes = [c_int, c_int, np.ctypeslib.ndpointer(dtype = np.float), np.ctypeslib.ndpointer(dtype = np.float)]
    lib.pdist_c.restype  = c_void_p
    
    n, p = xs.shape
    D = np.empty((n, n), np.float)
    
    lib.pdist_c(n, p, xs, D)
    return D

In [ ]:
print pdist_c(A)
%timeit pdist_c(xs)

### C++ 版本

C++ 的实现基本跟使用 C语言是一样的。只要增加一下额外的 C 语句，然后用合适的 C++ 编译器就可以了。

In [ ]:
%%file pdist_cpp.cpp
#include <cmath>

extern "C" 

// Variable length arrays are OK for C99 but not legal in C++
// void pdist_cpp(int n, int p, double xs[n*p], double D[n*n]) {
void pdist_cpp(int n, int p, double *xs, double *D) {
    for (int i=0; i<n; i++) {
        for (int j=0; j<n; j++) {
            double s = 0.0;
            for (int k=0; k<p; k++) {
                double tmp = xs[i*p+k] - xs[j*p+k];
                s += tmp*tmp;
            }
            D[i*n+j] = sqrt(s);
        }
    }
}

In [ ]:
# Compile to a shared library
! g++ -O3 -bundle -undefined dynamic_lookup pdist_cpp.cpp -o pdist_cpp.so
# Linux: 
# ! g++ -O3 -fPIC -shared pdist_cpp.cpp -o pdist_cpp.so

In [ ]:
from ctypes import CDLL, c_int, c_void_p

def pdist_cpp(xs):

    # Use ctypes to load the library
    lib = CDLL('./pdist_cpp.so')

    # We need to give the argument adn return types explicitly
    lib.pdist_cpp.argtypes = [c_int, c_int, np.ctypeslib.ndpointer(dtype = np.float), np.ctypeslib.ndpointer(dtype = np.float)]
    lib.pdist_cpp.restype  = c_void_p

    n, p = xs.shape
    D = np.empty((n, n), np.float)
    
    lib.pdist_cpp(n, p, xs, D)
    return D

In [ ]:
print pdist_cpp(A)
%timeit pdist_cpp(xs)

### Fortran 版本

In [ ]:
%%file pdist_fortran.f90

subroutine pdist_fortran (n, p, A, D)

    integer, intent(in) :: n
    integer, intent(in) :: p
    real(8), intent(in), dimension(n,p) :: A
    real(8), intent(inout), dimension(n,n) :: D
            
    integer :: i, j, k
    real(8) :: s, tmp
    ! note order of indices is different from C
    do j = 1, n
        do i = 1, n
            s = 0.0
            do k = 1, p
                tmp = A(i, k) - A(j, k)
                s = s + tmp*tmp
            end do
            D(i, j) = sqrt(s)
        end do
    end do
end subroutine

In [ ]:
! f2py -c -m flib pdist_fortran.f90 > /dev/null

In [ ]:
import flib
print flib.pdist_fortran.__doc__

In [ ]:
def pdist_fortran(xs):
    import flib
    n, p = xs.shape
    xs = np.array(xs, order='F')
    D = np.empty((n,n), np.float, order='F')
    flib.pdist_fortran(xs, D)
    return D

In [ ]:
print pdist_fortran(A)
%timeit pdist_fortran(xs)

In [ ]:
# Final bake-off 

w = 10
print 'Python'.ljust(w), 
%timeit pdist_python(xs)
print 'Numpy'.ljust(w), 
%timeit pdist_numpy(xs)
print 'Numexpr'.ljust(w), 
%timeit pdist_numexpr(xs)
print 'Numba'.ljust(w), 
%timeit pdist_numba(xs)
print 'Parakeet'.ljust(w), 
%timeit pdist_parakeet(xs)
print 'Cython'.ljust(w),
%timeit pdist_cython(xs)
print 'C'.ljust(w),
%timeit pdist_c(xs)
print 'C++'.ljust(w),
%timeit pdist_cpp(xs)
print 'Fortran'.ljust(w),
%timeit pdist_fortran(xs)

from scipy.spatial.distance import pdist as pdist_scipy
print 'Scipy'.ljust(w),
%timeit pdist_scipy(xs)

**最终优化（Final optimization）**: Scipy 只计算 i < j < n  这个范围的，因为成对的距离矩阵是对称的，所以只需要一般的时间就能完成。你试着自己修改一下咱们的 pdist_X 函数，也尽量利用一下这个对称性呢？

## 总结

* 使用 C, C++ 或者 Fortran 的性能表现基本一致
* 对于 JIT 解决方案:
    * Cython 是最快的，但是需要额外的精力花费在类型声明（type annotations）上；
    * numba 也差不多快，而且最简单好用，只考虑 jit 函数的话；
    * numexpr 稍微慢一点，而且最适合小规模的 numpy 表达式，不过也很方便使用。
* 纯粹 numpy 解决方案的性能也还不错，而且最简洁，在上文中只要一行就搞定
* 纯粹 python 方法性能太差，速度太慢，但是很有用，是去实现转换到原生代码或者进行 JIT 编译的模板
* 要注意到，最快速度解决方案相比纯 Python 方案快了几乎有 一千倍，n = 1000，p=3。

### 优化 Python 代码的建议

* 是否有一种已经存在的快速实现的方案呢？如果有，就不妨考虑使用；
* 可以先从 numpy/python 来构建原型，如果够快了，就可以了；
* 再看看如果通过 numpy 来进行向量化（vectoriazaiton）会不会更好；
* 接下来就要用到原生代码了（native code）:
    * 大多数 Python 开发者估计会用 Cython，Cython 也可以用来打包或者使用  C 或 C++ 的代码；
    * 使用 Numba 进行的即时编译（JIT compilation）发展迅速，可能不远的将来就会成为 Cython 的竞争对手；
    * 如果一个函数是"最小化的（minimal）"，通常就可以考虑使用 numrxpr 因为通常都不太费事就能搞定；
    * 如果你很熟悉 C/C++/Fortran 这些语言，当然也可以使用，这些都可以在 Python 中调用。
* 如果做近似估计（appropriate），可以考虑并行化（parallelization），后面会更细致讲这部分内容；
* 当你优化代码的时候，一定要记住：
    * 检查运算结果是否正确！
    * 多跑分评测（Profile often），因为通常很难在全局上估计一次优化的效果。
    * 时间宝贵，够快了就可以了，不要沉湎；
    * 如果弄一台更高配的机器就能搞定，那可能最好你就应该这么做了。

In [ ]:
%load_ext version_information

%version_information numpy, scipy, numexpr, numba, numbapro, parakeet, cython, f2py,